<a href="https://colab.research.google.com/github/WengGeyu/NMA-DL/blob/main/data_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git
%cd nnUNet
!pip install -e .

Cloning into 'nnUNet'...
remote: Enumerating objects: 10589, done.
remote: Counting objects: 100% (986/986), done.
remote: Compressing objects: 100% (435/435), done.
remote: Total 10589 (delta 653), reused 782 (delta 550), pack-reused 9603
Receiving objects: 100% (10589/10589), 5.38 MiB | 18.30 MiB/s, done.
Resolving deltas: 100% (8056/8056), done.
/content/nnUNet
Obtaining file:///content/nnUNet
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!nnUNetv2_plan_and_preprocess -d DATASET_ID --verify_dataset_integrity